分析流程: 讀資料--->資料可視化--->特徵處理--->切分資料--->訓練模型--->預測數據


修正
1.以迴圈方法將缺失值填入中位數避免極端值影響
2.以交叉驗證的方法

In [113]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [114]:
#load data
dengue_features_train = pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\dengue_features_train.csv")
dengue_features_test =  pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\dengue_features_test.csv")
dengue_labels_train = pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\dengue_labels_train.csv")

In [115]:
#see the shape of training feature
dengue_features_train.shape

(1456, 24)

In [116]:
#see the first 5 rows of training feature
dengue_features_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [117]:
#see the shape of test feature
dengue_features_test.shape

(416, 24)

In [118]:
#see the first 5 rows of test feature
dengue_features_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [119]:
#see the shape of label
dengue_labels_train.shape

(1456, 4)

In [120]:
#see the first 5 rows of test feature
dengue_labels_train.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [121]:
#merge features and labels of training data
df = pd.merge(dengue_features_train, dengue_labels_train, how='outer', on=dengue_labels_train.columns.tolist()[:-1])
df.head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6
5,sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,...,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1,10
9,sj,1990,27,1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,...,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1,6


In [122]:
df.shape

(1456, 25)

In [123]:
#see Nan
df.isnull().sum()

city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_mi

In [124]:
#將所有遺漏值以中位數方式填入，避免極端值造成的影響
for i in df.columns[4:25]:
    v = df[i].median()
    df[i] = df[i].fillna(value = v)

In [125]:
df.head(10) #檢查填入中位數後的資料樣貌

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6
5,sj,1990,23,1990-06-04,0.128817,0.174850,0.254314,0.181743,9.58,299.630000,...,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1,10
9,sj,1990,27,1990-07-02,0.128817,0.121550,0.160683,0.202567,14.41,300.154286,...,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1,6


In [126]:
df.isnull().sum() #檢查是否還有遺漏值

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

In [127]:
df.shape

(1456, 25)

In [128]:
#將資料切成train和test資料集
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation

data_train = df.iloc[:,4:24]
data_test = df.iloc[:,24]

train_X, test_X, train_y, test_y = cross_validation.train_test_split(data_train, data_test, test_size = 0.3)

In [129]:
#將訓練資料和測試資料進行標準化
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(train_X)
X_test_std = stdsc.fit_transform(test_X)

In [130]:
#建立線性回歸模型並將資料餵入
from sklearn.linear_model import Lasso, Ridge
clf = Lasso(alpha=1e0)
clf.fit(X_train_std, train_y)
print('Training accuracy:',clf.score(X_train_std,train_y))
print('Test accuracy:',clf.score(X_test_std,test_y))

Training accuracy: 0.121167724505
Test accuracy: 0.116971066911


In [131]:
print(clf.score(X_test_std, test_y))

0.116971066911


In [132]:
#用訓練好的模型來預測
y_pred = clf.predict(X_test_std)
y_pred=y_pred.round()
y_pred

array([ 13.,  42.,  43.,   3.,  41.,  43.,  25.,  12.,  27.,  22.,  14.,
        18.,  38.,  21.,  34.,  46.,  33.,  13.,  26.,  13.,  45.,  50.,
        17.,   2.,  17.,  37.,  39.,  23.,  31.,  20.,  42.,  50.,  39.,
        41.,  48.,  33.,  36.,  19.,   5.,  -2.,   1.,  29.,  40.,  28.,
        24.,  43.,  44.,  23.,  49.,  46.,  44.,  31.,  37.,  20.,  33.,
        27.,   8.,  40.,  42.,  16.,  13.,  48.,  34.,  44.,  13.,  12.,
        19.,  20.,  30.,   9.,  19.,   5.,  34.,  28.,   5.,  18.,  34.,
        48.,  43.,  39.,  38.,  44.,  19.,  30.,  47.,  19.,   6.,  33.,
        35.,   8.,  19.,   2.,  45.,  34.,  14.,  25.,  50.,  39.,  41.,
        21.,   5.,  13.,  18.,  -3.,  11.,  27.,  31.,  23.,   1.,  10.,
        13.,  19.,  41.,  23.,  43.,   7.,  18.,   4.,  45.,  -1.,   3.,
        15.,  40.,  43.,  50.,  23.,  21.,  35.,  31.,  20.,  44.,  30.,
        -1.,  42.,  23.,  12.,  18.,  41.,  22.,  45.,  41.,  19.,   9.,
        34.,   9.,   6.,  -1.,  10.,  16.,   5.,  4

In [133]:
y_pred[y_pred< 0] = 0
y_pred

array([ 13.,  42.,  43.,   3.,  41.,  43.,  25.,  12.,  27.,  22.,  14.,
        18.,  38.,  21.,  34.,  46.,  33.,  13.,  26.,  13.,  45.,  50.,
        17.,   2.,  17.,  37.,  39.,  23.,  31.,  20.,  42.,  50.,  39.,
        41.,  48.,  33.,  36.,  19.,   5.,   0.,   1.,  29.,  40.,  28.,
        24.,  43.,  44.,  23.,  49.,  46.,  44.,  31.,  37.,  20.,  33.,
        27.,   8.,  40.,  42.,  16.,  13.,  48.,  34.,  44.,  13.,  12.,
        19.,  20.,  30.,   9.,  19.,   5.,  34.,  28.,   5.,  18.,  34.,
        48.,  43.,  39.,  38.,  44.,  19.,  30.,  47.,  19.,   6.,  33.,
        35.,   8.,  19.,   2.,  45.,  34.,  14.,  25.,  50.,  39.,  41.,
        21.,   5.,  13.,  18.,   0.,  11.,  27.,  31.,  23.,   1.,  10.,
        13.,  19.,  41.,  23.,  43.,   7.,  18.,   4.,  45.,   0.,   3.,
        15.,  40.,  43.,  50.,  23.,  21.,  35.,  31.,  20.,  44.,  30.,
         0.,  42.,  23.,  12.,  18.,  41.,  22.,  45.,  41.,  19.,   9.,
        34.,   9.,   6.,   0.,  10.,  16.,   5.,  4

In [134]:
#以MAE評估模型效能
from sklearn.metrics import mean_absolute_error, r2_score
# The coefficients
print('Coefficients: \n', clf.coef_)

# The mean absolute error
print("Mean absolute error: %.2f"
      % mean_absolute_error(test_y, y_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(test_y, y_pred))


Coefficients: 
 [  0.           0.83215234   0.           0.          -0.           6.14738324
   0.           0.          -0.           0.           0.          -0.
  -0.38492463   0.         -11.11068623   1.20801713  -1.28839044
   3.34741668   1.18603917   0.        ]
Mean absolute error: 18.70
Variance score: 0.12


In [135]:
dengue_features_test_df = pd.DataFrame(dengue_features_test) #將要預測的資料轉換成dataframe的格式
dengue_features_test_df.isnull().sum() #檢查是否還有遺漏值

city                                      0
year                                      0
weekofyear                                0
week_start_date                           0
ndvi_ne                                  43
ndvi_nw                                  11
ndvi_se                                   1
ndvi_sw                                   1
precipitation_amt_mm                      2
reanalysis_air_temp_k                     2
reanalysis_avg_temp_k                     2
reanalysis_dew_point_temp_k               2
reanalysis_max_air_temp_k                 2
reanalysis_min_air_temp_k                 2
reanalysis_precip_amt_kg_per_m2           2
reanalysis_relative_humidity_percent      2
reanalysis_sat_precip_amt_mm              2
reanalysis_specific_humidity_g_per_kg     2
reanalysis_tdtr_k                         2
station_avg_temp_c                       12
station_diur_temp_rng_c                  12
station_max_temp_c                        3
station_min_temp_c              

In [145]:
#將所有遺漏值以中位數方式填入，避免極端值造成的影響
for j in dengue_features_test_df.columns[4:]:
    vv = dengue_features_test_df[j].median()
    dengue_features_test_df[j] = dengue_features_test_df[j].fillna(value = vv)

In [146]:
dengue_features_test_df_new = dengue_features_test_df.iloc[:,4:24] #略過前四欄的字串資料

In [147]:
dengue_features_test_df_new
dengue_features_test_df_new_std = stdsc.fit_transform(dengue_features_test_df_new)

In [163]:
#進行預測
result = clf.predict(dengue_features_test_df_new_std)
result[result< 0] = 0
result = result.round()
result

array([ 29.75239638,  27.10733025,  32.06446687,  35.62963583,
        37.23359161,  39.85568362,  35.20697959,  42.86989975,
        44.070689  ,  44.06502123,  40.73651614,  44.62102666,
        47.33992933,  45.35623102,  44.06040754,  49.97155863,
        46.3709111 ,  46.11874981,  49.29953726,  48.58104149,
        43.83296509,  38.90538922,  43.92039229,  42.50037296,
        39.10296161,  36.97756758,  40.22208743,  38.66993972,
        35.64592307,  35.17746446,  33.57688159,  25.15357924,
        27.63266502,  29.92979484,  19.96732353,  32.88978747,
        19.00975191,  27.00406952,  26.29710365,  27.54706166,
        23.22404723,  24.16799959,  22.13736552,  23.13083391,
        22.10915081,  23.60264207,  16.20103667,  15.25112606,
        17.2131013 ,  21.26571911,  25.15111544,  30.15433627,
        31.84594743,  25.68808606,  31.56612844,  27.57392031,
        34.66018243,  40.15436478,  38.02899599,  41.49438749,
        42.44655902,  45.07678695,  47.29873258,  47.13

In [160]:
#讀入比賽格式
dengue_submission = pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\submission_format.csv")

In [161]:
#將預測結果寫入比賽格式
dengue_submission['total_cases'] = result
dengue_submission['total_cases'] = dengue_submission['total_cases'].astype(int)
dengue_submission.to_csv('submission_format_v3.csv', index = False)

In [162]:
dengue_submission.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,30
1,sj,2008,19,27
2,sj,2008,20,32
3,sj,2008,21,36
4,sj,2008,22,37
